In [1]:
from collections import Counter
from dataclasses import dataclass
import random
from typing import List, Dict, Tuple

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
# import pytorch_lightning as pl
from sklearn.decomposition import PCA
# import torch
# from torch import nn
# import torch.multiprocessing
# from torch.utils.data import DataLoader, Dataset
import tqdm

GLOBAL_SEED = 42  # number of life
# torch.manual_seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

In [2]:
order_df = pd.read_csv("./data/order_products__prior.csv")
order_df

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [3]:
def get_list_orders(order_df: pd.DataFrame) -> List[List[int]]:
    order_df = order_df.sort_values(by=["order_id", "add_to_cart_order"])
    return order_df.groupby("order_id")["product_id"].apply(list).tolist()

all_orders = get_list_orders(order_df)
print(f"Number of orders: {len(all_orders)}")
print(f"First 3 orders: {all_orders[:3]}")

Number of orders: 3214874
First 3 orders: [[33120, 28985, 9327, 45918, 30035, 17794, 40141, 1819, 43668], [33754, 24838, 17704, 21903, 17668, 46667, 17461, 32665], [46842, 26434, 39758, 27761, 10054, 21351, 22598, 34862, 40285, 17616, 25146, 32645, 41276]]


In [4]:
min_product_per_order = 2
orders = [order for order in all_orders if len(order) >= min_product_per_order]
print(f"Number of orders with at least {min_product_per_order} products: {len(orders)}")

Number of orders with at least 2 products: 3058126


In [5]:
product_df = pd.read_csv("./data/products.csv", usecols=["product_id", "product_name"])
product_df.head(10)
print(len(product_df))

49688


In [6]:
# creat a mapping between product_id and product_name
product_name_by_id = product_df.set_index("product_id").to_dict()["product_name"]
print(f"Number of product: {len(product_name_by_id)}")
product_name_by_id

Number of product: 49688


{1: 'Chocolate Sandwich Cookies',
 2: 'All-Seasons Salt',
 3: 'Robust Golden Unsweetened Oolong Tea',
 4: 'Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce',
 5: 'Green Chile Anytime Sauce',
 6: 'Dry Nose Oil',
 7: 'Pure Coconut Water With Orange',
 8: "Cut Russet Potatoes Steam N' Mash",
 9: 'Light Strawberry Blueberry Yogurt',
 10: 'Sparkling Orange Juice & Prickly Pear Beverage',
 11: 'Peach Mango Juice',
 12: 'Chocolate Fudge Layer Cake',
 13: 'Saline Nasal Mist',
 14: 'Fresh Scent Dishwasher Cleaner',
 15: 'Overnight Diapers Size 6',
 16: 'Mint Chocolate Flavored Syrup',
 17: 'Rendered Duck Fat',
 18: 'Pizza for One Suprema  Frozen Pizza',
 19: 'Gluten Free Quinoa Three Cheese & Mushroom Blend',
 20: 'Pomegranate Cranberry & Aloe Vera Enrich Drink',
 21: 'Small & Medium Dental Dog Treats',
 22: 'Fresh Breath Oral Rinse Mild Mint',
 23: 'Organic Turkey Burgers',
 24: 'Tri-Vi-Sol® Vitamins A-C-and D Supplement Drops for Infants',
 25: 'Salted Caramel Lean Protein & 

In [7]:
# All products appearing in orders
ordered_products = set([product for order in orders for product in order])
print(ordered_products)
product_mapping = dict()
# build mappings: product_id -> product_index, product_index -> product_name
product_mapping["index_by_id"] = dict()
product_mapping["name_by_index"] = dict()
ind = 0
for ind, product_id in enumerate(ordered_products):
    product_name = product_name_by_id[product_id]
    product_mapping["index_by_id"][product_id] = ind
    product_mapping["name_by_index"][ind] = product_name

product_mapping

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

{'index_by_id': {1: 0,
  2: 1,
  3: 2,
  4: 3,
  5: 4,
  6: 5,
  7: 6,
  8: 7,
  9: 8,
  10: 9,
  11: 10,
  12: 11,
  13: 12,
  14: 13,
  15: 14,
  16: 15,
  17: 16,
  18: 17,
  19: 18,
  20: 19,
  21: 20,
  22: 21,
  23: 22,
  24: 23,
  25: 24,
  26: 25,
  27: 26,
  28: 27,
  29: 28,
  30: 29,
  31: 30,
  32: 31,
  33: 32,
  34: 33,
  35: 34,
  36: 35,
  37: 36,
  38: 37,
  39: 38,
  40: 39,
  41: 40,
  42: 41,
  43: 42,
  44: 43,
  45: 44,
  46: 45,
  47: 46,
  48: 47,
  49: 48,
  50: 49,
  51: 50,
  52: 51,
  53: 52,
  54: 53,
  55: 54,
  56: 55,
  57: 56,
  58: 57,
  59: 58,
  60: 59,
  61: 60,
  62: 61,
  63: 62,
  64: 63,
  65: 64,
  66: 65,
  67: 66,
  68: 67,
  69: 68,
  70: 69,
  71: 70,
  72: 71,
  73: 72,
  74: 73,
  75: 74,
  76: 75,
  77: 76,
  78: 77,
  79: 78,
  80: 79,
  81: 80,
  82: 81,
  83: 82,
  84: 83,
  85: 84,
  86: 85,
  87: 86,
  88: 87,
  89: 88,
  90: 89,
  91: 90,
  92: 91,
  93: 92,
  94: 93,
  95: 94,
  96: 95,
  97: 96,
  98: 97,
  99: 98,
  100: 99,
  1

In [8]:
indexed_orders = [
    [product_mapping["index_by_id"][product_id] for product_id in order]
    for order in orders
]

indexed_orders
# print(len(indexed_orders))

[[33113, 28978, 9323, 45907, 30028, 17790, 40131, 1818, 43658],
 [33747, 24834, 17700, 21899, 17664, 46654, 17457, 32658],
 [46829,
  26428,
  39749,
  27754,
  10050,
  21347,
  22594,
  34855,
  40275,
  17612,
  25142,
  32638,
  41266],
 [13172,
  15001,
  47316,
  27959,
  23905,
  48356,
  13241,
  9629,
  27354,
  6345,
  40868,
  6181,
  47988,
  20910,
  37003,
  12958,
  45687,
  24769,
  18565,
  41166,
  48352,
  47196,
  46510,
  38684,
  48811,
  8475],
 [40452, 15869, 41887],
 [34043, 46789],
 [21401,
  47876,
  11178,
  2013,
  29186,
  34196,
  14988,
  31499,
  23284,
  44522,
  18358,
  27360,
  431,
  3987,
  14179],
 [24848,
  4793,
  31710,
  47752,
  4602,
  1528,
  21133,
  22118,
  34127,
  27150,
  14988,
  49221,
  26836,
  3463,
  25715],
 [30155, 27079, 5991, 1312, 31499],
 [30590,
  15217,
  43761,
  37877,
  37207,
  34328,
  3163,
  26904,
  38879,
  38041,
  25209,
  11171,
  23386,
  29464,
  43501],
 [17326,
  27401,
  35412,
  195,
  44624,
  26872,


In [9]:
context_window = 5
# total number of context products, including positive and negative products
all_targets = []
all_positive_contexts = []
for order in indexed_orders:
    for i, product in enumerate(order):
        all_targets.append(product)
        positive_context = [
            order[j]
            for j in range(
                max(0, i - context_window), min(len(order), i + context_window + 1)
            )
            if j != i
        ]
        all_positive_contexts.append(positive_context)

print("Sample order:", indexed_orders[0])
for i in range(10):
    print(f"Target product: {all_targets[i]}", end = ", ")
    print(f"Positive context products: {all_positive_contexts[i]}")

Sample order: [33113, 28978, 9323, 45907, 30028, 17790, 40131, 1818, 43658]
Target product: 33113, Positive context products: [28978, 9323, 45907, 30028, 17790]
Target product: 28978, Positive context products: [33113, 9323, 45907, 30028, 17790, 40131]
Target product: 9323, Positive context products: [33113, 28978, 45907, 30028, 17790, 40131, 1818]
Target product: 45907, Positive context products: [33113, 28978, 9323, 30028, 17790, 40131, 1818, 43658]
Target product: 30028, Positive context products: [33113, 28978, 9323, 45907, 17790, 40131, 1818, 43658]
Target product: 17790, Positive context products: [33113, 28978, 9323, 45907, 30028, 40131, 1818, 43658]
Target product: 40131, Positive context products: [28978, 9323, 45907, 30028, 17790, 1818, 43658]
Target product: 1818, Positive context products: [9323, 45907, 30028, 17790, 40131, 43658]
Target product: 43658, Positive context products: [45907, 30028, 17790, 40131, 1818]
Target product: 33747, Positive context products: [24834, 17

In [10]:
def get_sampling_weights(orders):
    product_freq = Counter([product for order in orders for product in order])
    print(product_freq)
    sampling_weights = [0 for _ in product_freq]
    for product_index, count in product_freq.items():
        sampling_weights[product_index] = count**0.5
    return sampling_weights

sampling_weights = get_sampling_weights(indexed_orders)

len(sampling_weights)

Counter({24848: 470518, 13172: 376676, 21133: 263563, 21899: 240755, 47196: 212887, 47752: 176303, 47613: 152266, 16793: 141933, 26203: 140328, 27838: 136917, 27959: 136695, 22931: 113150, 24960: 109539, 44996: 104638, 39266: 99435, 49668: 97095, 28197: 89414, 5873: 87537, 8273: 84807, 40696: 84151, 4917: 82373, 30384: 80292, 45055: 79597, 42255: 76645, 44621: 75499, 49221: 75443, 19053: 74984, 4602: 73069, 37638: 72717, 21612: 72634, 17790: 72611, 27098: 71485, 30482: 71208, 31710: 69409, 27080: 68906, 44348: 67992, 28978: 67336, 46966: 67082, 8514: 66907, 41940: 64164, 26598: 61569, 5074: 60699, 34119: 60491, 22031: 59607, 39867: 58617, 35944: 57576, 10745: 57430, 43342: 56923, 9072: 55753, 21934: 55420, 43950: 55302, 24180: 55217, 34962: 54312, 19656: 54022, 46654: 51959, 48665: 51862, 39918: 50075, 31499: 50001, 25884: 49978, 24834: 49370, 12337: 49303, 5447: 48909, 22821: 48864, 5782: 47639, 35214: 46393, 28835: 45455, 33724: 45137, 27514: 42997, 44131: 42922, 8170: 42658, 20110: 

49674

In [11]:
class ProductSampler:
    def __init__(self, products, weights, pre_drawn=10_000_000):
        self.products = products
        self.weights = weights
        self.pre_drawn = pre_drawn
        self.pre_drawn_products = []

    def refill(self):
        self.pre_drawn_products = random.choices(
            population=self.products, weights=self.weights, k=self.pre_drawn
        )

    def draw(self):
        if not self.pre_drawn_products:
            self.refill()
        return self.pre_drawn_products.pop()


num_products = len(ordered_products)
product_sampler = ProductSampler(
    products=range(num_products),
    weights=sampling_weights,
    pre_drawn=10_000_000,
)

print("Sampling samples:", [product_sampler.draw() for _ in range(10)])

Sampling samples: [23012, 19504, 25757, 30449, 49168, 36892, 43361, 23140, 31710, 22609]


In [12]:
import tensorflow as tf
import numpy as np

class TargetContextDataset(tf.keras.utils.Sequence):
    def __init__(self, all_targets, all_positive_contexts, product_sampler, num_context_products=10, batch_size=8192, shuffle=True):
        self.all_targets = all_targets
        self.all_positive_contexts = all_positive_contexts
        self.product_sampler = product_sampler
        self.num_context_products = num_context_products
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.all_targets))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __len__(self):
        return len(self.all_targets) // self.batch_size

    def __getitem__(self, idx):
        indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_targets = []
        batch_contexts = []
        batch_masks = []

        for index in indexes:
            target = np.array([self.all_targets[index]])
            positive_contexts = self.all_positive_contexts[index].copy()
            num_pos = len(positive_contexts)
            num_neg = self.num_context_products - len(positive_contexts)
            mask = [1] * num_pos + [0] * num_neg
            while len(positive_contexts) < self.num_context_products:
                product = self.product_sampler.draw()
                if product not in positive_contexts:
                    positive_contexts.append(product)

            contexts = np.array(positive_contexts)
            mask = np.array(mask)

            batch_targets.append(target)
            batch_contexts.append(contexts)
            batch_masks.append(mask)

        return np.array(batch_targets), np.array(batch_contexts), np.array(batch_masks)

# Assuming all_targets, all_positive_contexts, product_sampler are defined elsewhere
training_data = TargetContextDataset(all_targets, all_positive_contexts, product_sampler, num_context_products=10)


# Example usage:
for i in range(len(training_data)):
    batch_targets, batch_contexts, batch_masks = training_data[i]
    print("Batch Targets:", batch_targets.shape)
    print("Batch Contexts:", batch_contexts.shape)
    print("Batch Masks:", batch_masks.shape)

    print("Target:", batch_targets[0])
    print("Contexts:", batch_contexts[0])
    print("Mask:", batch_masks[0])
    break  # break after the first batch for demonstration


Batch Targets: (8192, 1)
Batch Contexts: (8192, 10)
Batch Masks: (8192, 10)
Target: [38881]
Contexts: [47752 33287 22155 27407 20278 25819 30482 28053 44236 14100]
Mask: [1 1 1 1 1 1 1 0 0 0]


In [13]:
import tensorflow as tf

class TargetContextDataset:
    def __init__(self, all_targets, all_positive_contexts, product_sampler, num_context_products=10):
        self.all_targets = all_targets
        self.all_positive_contexts = all_positive_contexts
        self.product_sampler = product_sampler
        self.num_context_products = num_context_products

    def __len__(self):
        return len(self.all_targets)

    def __getitem__(self, index):
        target = [self.all_targets[index]]
        positive_contexts = self.all_positive_contexts[index].copy()
        num_pos = len(positive_contexts)
        num_neg = self.num_context_products - num_pos
        mask = [1.0] * num_pos + [0.0] * num_neg
        
        while len(positive_contexts) < self.num_context_products:
            product = self.product_sampler.draw()
            if product not in positive_contexts:
                positive_contexts.append(product)

        return target, positive_contexts, mask

    def generator(self):
        for i in range(len(self)):
            yield self[i]

# Assuming all_targets, all_positive_contexts, and product_sampler are defined
dataset = TargetContextDataset(all_targets, all_positive_contexts, product_sampler)

# Convert to tf.data.Dataset
tf_dataset = tf.data.Dataset.from_generator(
    dataset.generator,
    output_types=(tf.int32, tf.int32, tf.float32),
    output_shapes=((1,), (dataset.num_context_products,), (dataset.num_context_products,))
)


Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead


In [14]:
# Set the parameters for batching and shuffling
batch_size = 8192
tf_dataset = tf_dataset.shuffle(buffer_size=10000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

# Iterate over the dataset
for target, context_products, labels in tf_dataset.take(1):  # Using take(1) to simulate the break in your PyTorch loop
    print("Target:", target[0])
    print("Context products:", context_products[0])
    print("Labels:", labels[0])


Target: tf.Tensor([9174], shape=(1,), dtype=int32)
Context products: tf.Tensor([45736 19318 34237  7171 31499 41940 21133 13834 33709  4459], shape=(10,), dtype=int32)
Labels: tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1. 1. 1.], shape=(10,), dtype=float32)


In [15]:
import tensorflow as tf

class SigmoidBCELoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, (y_pred.shape[0], -1))
        loss = tf.keras.losses.binary_crossentropy(y_true, y_pred, from_logits=True)
        return tf.reduce_mean(loss)

# Create an instance of the loss function
loss_fn = SigmoidBCELoss()

sample_logits = tf.constant([[100.0, -100.0], [1.0, 1.0]])
sample_labels = tf.constant([[1.0, 0.0], [1.0, 0.0]])

loss_value = loss_fn(sample_labels, sample_logits)
print("Loss:", loss_value.numpy())


Loss: 0.40663087


In [16]:
import tensorflow as tf

class Prod2VecModel(tf.keras.Model):
    def __init__(self, num_products, embed_size=50):
        super().__init__()
        self.embed_size = embed_size
        self.embed_t = tf.keras.layers.Embedding(input_dim=num_products, output_dim=self.embed_size)
        self.embed_c = tf.keras.layers.Embedding(input_dim=num_products, output_dim=self.embed_size)

    def call(self, inputs, training=False):
        targets, contexts = inputs
        v = self.embed_t(targets)
        u = self.embed_c(contexts)
        pred = tf.linalg.matmul(v, tf.transpose(u, perm=[0, 2, 1]))
        return pred


In [17]:
num_products = 10000  # You need to set this based on your actual number of products
embed_size = 100
model = Prod2VecModel(num_products, embed_size)

# Assuming SigmoidBCELoss is already defined, or using the built-in loss
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3),
              loss=loss_fn,
              metrics=['accuracy'])  # Add any metrics if needed


In [18]:
import numpy as np

# Example data generation
num_samples = 1000
targets = np.random.randint(0, num_products, size=(num_samples, 1))
contexts = np.random.randint(0, num_products, size=(num_samples, 10))
labels = np.random.random(size=(num_samples, 1, 10))

train_dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)


In [19]:
# Train the model
with tf.device('/gpu:0'):  # Use '/cpu:0' if GPU is not available
    model.fit(train_dataset, epochs=50)


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.0946 - loss: 0.6932
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5394 - loss: 0.6771
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5667 - loss: 0.6397
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6385 - loss: 0.5801
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7059 - loss: 0.5308
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8394 - loss: 0.5124
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9128 - loss: 0.5075
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9471 - loss: 0.5029
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9675 - loss: 0.5027
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9729 - loss: 0.5023
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9719 - loss: 0.5012
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9839 - lo

In [20]:
# Extract embeddings from the TensorFlow model
embs_arr = model.embed_t.get_weights()[0]


In [21]:
import numpy as np
from scipy.spatial import distance

class NearestNeighbor:
    def __init__(self, embeddings, measure='cosine'):
        self.embeddings = embeddings
        self.measure = measure

    def find_nearest_neighbors(self, vector, k=2):
        if self.measure == 'cosine':
            # Compute cosine distances between the vector and all other vectors in the embeddings
            distances = distance.cdist([vector], self.embeddings, 'cosine')[0]
        else:
            # Euclidean distances can also be used
            distances = distance.cdist([vector], self.embeddings, 'euclidean')[0]
        
        # Get the indices of the smallest k distances
        nearest_indices = np.argsort(distances)[:k]
        return nearest_indices


In [25]:
# Example to find and display similar items
names = [product_mapping["name_by_index"][i] for i in range(num_products)]

sub_name = "Chocolate"
ids = [ind for ind, name in enumerate(names) if sub_name in name]

# Initialize Nearest Neighbor search
emb_nn = NearestNeighbor(embs_arr, measure="cosine")

for ind in ids[:5]:
    print('==========')
    print(f'Similar items of "{names[ind]}":')
    nearest_ids = emb_nn.find_nearest_neighbors(embs_arr[ind], k=5)
    print([names[i] for i in nearest_ids])


Similar items of "Chocolate Sandwich Cookies":
['Chocolate Sandwich Cookies', 'Calcium Plus Vitamin D Vegetable Oil Spread', 'Tandoori Chicken Samosa', 'Calendula Baby Bath Wash', 'SpongeBob Shapes Macaroni & Cheese Dinner']
Similar items of "Chocolate Fudge Layer Cake":
['Chocolate Fudge Layer Cake', 'Creamy Chicken & Dumplings Soup', 'Super Slimming Herbal Tea', 'Scallions', 'Gift With Flask']
Similar items of "Mint Chocolate Flavored Syrup":
['Mint Chocolate Flavored Syrup', 'Hoisen Sauce', 'Pear and Blueberry Baby Breakfast', 'Toasted Onion Dip Mix', 'Cotto Salami']
Similar items of "Coconut Chocolate Chip Energy Bar":
['Coconut Chocolate Chip Energy Bar', 'Light Blueberry Patch/Blackberry Pomegranate Variety Pack Fat Free Yogurt', "Ibuprofen Infants' Berry Pain Reliever/Fever Reducer", 'Free Range Organic Brown Large Egg', 'Pill Pouches Chicken Flavor']
Similar items of "Vanilla Milk Chocolate Almond Ice Cream Bars Multi-Pack":
['Vanilla Milk Chocolate Almond Ice Cream Bars Multi-